# STAC API

## A short word on STAC

STAC is the central way of accessing any spatio-temporal data on terrabyte. See here for an introduction and the detailed sepcification:
* https://stacspec.org/en
* https://github.com/radiantearth/stac-spec

Principally, data is offered over a *catalog* containing data from various sources. This catalog is further sub-divided into *collections*. 
A collection could for example contain a certain satellite data product like Sentinel-1 GRD, SLC or Sentinel-2 L2A. 
Each collection consists of multiple items, which might represent individual satellite scenes or product tiles (e.g. the MGRS tiles of Sentinel-2). 
Each item consists of one or many *assets*, which contain links to the actual data. For example individual GeoTIFF files for each band.

The terrabyte STAC catalog URL is https://stac.terrabyte.lrz.de/public/api.

Further down you will find examples on how to list all available collections and how to query the STAC catalog in Python using the pystac library. The content can also be explored by opening the above link in a browser.

![stac-catalog.png](_images/stac-catalog.png)

From there you can navigate to individual collections and explore their metadata content for search queries. For example, under https://stac.terrabyte.lrz.de/public/api/collections/sentinel-2-c1-l2a/items → `features` → `0` → `properties` we can list the properties of the first item of the sentinel-2-c1-l2a collection. This list contains, amongst others, the entries `eo:cloud_cover` and `s2:mgrs_tile`, which will be queried in the tutorial below. The namespaces eo, grid, proj, mgrs, sat and view refer to dedicated STAC extensions. s2 is a custom namespace for expressing Sentinel-2 specific metadata that cannot be described in other STAC extensions.

![stac-collection.png](_images/stac-collection.png)

## Explore the STAC catalog

In [1]:
from pystac_client import Client as pystacclient
catalog = pystacclient.open('https://stac.terrabyte.lrz.de/public/api')
 
# list the IDs of all STAC catalog collections
for collection in catalog.get_all_collections():
    print( collection.id)

cop-dem-glo-90
modis-13q1-061
viirs-09ga-001
modis-09ga-061
modis-13a3-061
modis-13a2-061
landsat-etm-c2-l2
modis-09gq-061
modis-10a1-061
landsat-tm-c2-l2
landsat-ot-c2-l2
sentinel-2-c0-l2a
sentinel-2-c0-l1c
cop-dem-glo-30
viirs-15a2h-001
viirs-13a1-001
sentinel-1-nrb
sentinel-1-slc
sentinel-1-grd
sentinel-2-c1-l2a


## Query data from the STAC catalog

gte and lte describe greater than or equal and lower than or equal, respectively.

In [8]:
start = '2022-01-01T00:00:00Z'
end = '2022-12-31T23:59:59Z'
collection = 'sentinel-2-c1-l2a'

query = {
    'eo:cloud_cover': {
        "gte": 0,
        "lte": 30
    },
    's2:mgrs_tile': {'eq': '33UUP'}
}

results = catalog.search(
    collections=[collection],
    datetime=[start, end],
    query=query, 
)
items = results.item_collection()
print("%s items found" % len(items))

62 items found


## Show first item

In [9]:
items[0]

<Item id=S2A_MSIL2A_20221227T102431_R065_T33UUP_20221227T140052>

## List items as Pandas GeoDataFrame

In [10]:
import geopandas as gpd
dataframe = gpd.GeoDataFrame.from_features(items)
dataframe

,geometry,created,datetime,platform,grid:code,proj:epsg,providers,instruments,esa:scene_id,s2:mgrs_tile,...,s2:cloud_shadow_percentage,s2:nodata_pixel_percentage,s2:unclassified_percentage,s2:dark_features_percentage,s2:not_vegetated_percentage,s2:degraded_msi_data_percentage,s2:high_proba_clouds_percentage,s2:reflectance_conversion_factor,s2:medium_proba_clouds_percentage,s2:saturated_defective_pixel_percentage
0,"POLYGON ((12.60114 48.72640, 12.59859 48.72096...",2023-07-17T21:21Z,2022-12-27T10:24:31.024000Z,sentinel-2a,MGRS-33UUP,32633,[{'url': 'https://earth.esa.int/web/guest/home...,[msi],S2A_MSIL2A_20221227T102431_N0509_R065_T33UUP_2...,33UUP,...,0.029668,93.256605,0.123643,0.403402,34.171543,0.0037,0.011415,1.033683,0.005609,0.0
1,"POLYGON ((12.60263 48.72643, 12.57872 48.67529...",2023-07-17T21:22Z,2022-12-17T10:24:31.024000Z,sentinel-2a,MGRS-33UUP,32633,[{'url': 'https://earth.esa.int/web/guest/home...,[msi],S2A_MSIL2A_20221217T102431_N0509_R065_T33UUP_2...,33UUP,...,0.000000,93.207598,0.009183,0.125096,1.860314,0.0000,0.314326,1.032036,0.568230,0.0
2,"POLYGON ((13.10898 47.74726, 13.10952 47.74884...",2023-07-17T21:22Z,2022-10-27T10:00:29.024000Z,sentinel-2b,MGRS-33UUP,32633,[{'url': 'https://earth.esa.int/web/guest/home...,[msi],S2B_MSIL2A_20221027T100029_N0400_R122_T33UUP_2...,33UUP,...,0.000000,67.028844,0.044568,2.360363,17.604588,0.0000,0.001942,1.010958,0.103788,0.0
3,"POLYGON ((12.28064 48.72091, 13.77299 48.74648...",2023-07-17T21:22Z,2022-10-25T10:11:11.024000Z,sentinel-2a,MGRS-33UUP,32633,[{'url': 'https://earth.esa.int/web/guest/home...,[msi],S2A_MSIL2A_20221025T101111_N0400_R022_T33UUP_2...,33UUP,...,8.834014,0.001158,6.388278,0.553050,15.764636,0.0000,9.132036,1.009851,13.965878,0.0
4,"POLYGON ((12.28064 48.72091, 13.77299 48.74648...",2023-07-17T21:22Z,2022-10-20T10:09:39.024000Z,sentinel-2b,MGRS-33UUP,32633,[{'url': 'https://earth.esa.int/web/guest/home...,[msi],S2B_MSIL2A_20221020T100939_N0400_R022_T33UUP_2...,33UUP,...,0.949072,0.000050,0.167750,0.545616,19.083035,0.0000,1.338682,1.007030,4.548574,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,"POLYGON ((13.11731 47.74740, 13.13818 47.80459...",2023-07-17T21:12Z,2022-02-14T10:01:21.024000Z,sentinel-2a,MGRS-33UUP,32633,[{'url': 'https://earth.esa.int/web/guest/home...,[msi],S2A_MSIL2A_20220214T100121_N0400_R122_T33UUP_2...,33UUP,...,0.008273,67.276353,0.060094,3.249630,33.941147,0.0344,0.000760,1.027025,0.039775,0.0
58,"POLYGON ((12.28064 48.72091, 13.77299 48.74648...",2023-07-17T21:12Z,2022-02-12T10:10:29.024000Z,sentinel-2b,MGRS-33UUP,32633,[{'url': 'https://earth.esa.int/web/guest/home...,[msi],S2B_MSIL2A_20220212T101029_N0400_R022_T33UUP_2...,33UUP,...,6.021328,0.006672,1.228280,1.334959,36.369061,0.0000,0.717657,1.027739,2.137687,0.0
59,"POLYGON ((12.60113 48.72640, 12.57872 48.67827...",2023-07-17T21:12Z,2022-02-10T10:21:51.024000Z,sentinel-2a,MGRS-33UUP,32633,[{'url': 'https://earth.esa.int/web/guest/home...,[msi],S2A_MSIL2A_20220210T102151_N0400_R065_T33UUP_2...,33UUP,...,0.069921,93.257153,0.080647,0.034641,52.814668,0.0000,0.028441,1.028419,0.054323,0.0
60,"POLYGON ((13.10806 47.74724, 13.13614 47.82336...",2023-07-17T21:12Z,2022-02-09T10:00:49.024000Z,sentinel-2b,MGRS-33UUP,32633,[{'url': 'https://earth.esa.int/web/guest/home...,[msi],S2B_MSIL2A_20220209T100049_N0400_R122_T33UUP_2...,33UUP,...,0.243259,67.037010,0.635033,3.592503,25.425598,0.0000,14.310537,1.028753,4.338174,0.0


## Visualize the covered area

In [11]:
import folium
import folium.plugins as folium_plugins

map = folium.Map(tiles='Stamen Terrain')
layer_control = folium.LayerControl(position='topright', collapsed=True)
fullscreen = folium_plugins.Fullscreen()
style = {'fillColor': '#00000000', "color": "#0000ff", "weight": 1}

footprints = folium.GeoJson(
    dataframe.to_json(),
    name='Stac Item footprints',
    style_function=lambda x: style,
    control=True
)

footprints.add_to(map)
layer_control.add_to(map)
fullscreen.add_to(map)
map.fit_bounds(map.get_bounds())
map